#### Objective : To try hand at feature engineering 
- learn the importance 
- some techniques


Dataset : https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data 

In [125]:
import numpy as np

import pandas as pd
df_train = pd.read_csv(r'./Data/feature_engineering/train.csv',index_col='id')

In [126]:
print(df_train.info())
print(df_train.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 159571 entries, 0000997932d777bf to fff46fc426af1f9a
Data columns (total 7 columns):
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(1)
memory usage: 9.7+ MB
None
               toxic   severe_toxic        obscene         threat  \
count  159571.000000  159571.000000  159571.000000  159571.000000   
mean        0.095844       0.009996       0.052948       0.002996   
std         0.294379       0.099477       0.223931       0.054650   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       0.000000   
75%         0.000000       0.000000       0.000000       0

In [127]:
# checking the null values
print(df_train.isnull().sum())

comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64


In [128]:
df_train.tail()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [129]:
df_train['comment_text'].tail()

id
ffe987279560d7ff    ":::::And for the second time of asking, when ...
ffea4adeee384e90    You should be ashamed of yourself \n\nThat is ...
ffee36eab5c267c9    Spitzer \n\nUmm, theres no actual article for ...
fff125370e4aaaf3    And it looks like it was actually you who put ...
fff46fc426af1f9a    "\nAnd ... I really don't think you understand...
Name: comment_text, dtype: object

In [130]:
df_train.loc[:,['toxic','severe_toxic','obscene','threat','insult','identity_hate']].apply(sum, axis=0)

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [131]:
#CLEANING UP THE TEXT
#Function to clean up the text
def standardizetext(df, textfield):
    df[textfield] = df[textfield].str.replace(r"http\S+", "")
    df[textfield] = df[textfield].str.replace(r"http", "")
    df[textfield] = df[textfield].str.replace(r"@\S+", "")
    df[textfield] = df[textfield].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[textfield] = df[textfield].str.replace(r"@", "at")
    df[textfield] = df[textfield].str.lower()
    return df

In [172]:
df_train = standardizetext(df_train, "comment_text")
df_train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,total_length,capitals,num_exclamation_marks,num_question_marks,num_punctuation,num_symbols,num_words,num_unique_words,words_vs_unique,num_smilies
id,,,,,,,,,,,,,,,,,
0000997932d777bf,explanation\nwhy the edits made under my usern...,0,0,0,0,0,0,264,0,0,1,1,0,47,45,0.957447,0
000103f0d9cfb60f,d'aww! he matches this background colour i'm s...,0,0,0,0,0,0,112,0,1,0,2,0,18,18,1.000000,0
000113f07ec002fd,"hey man, i'm really not trying to edit war it...",0,0,0,0,0,0,233,0,0,0,1,0,42,39,0.928571,0
0001b41b1c6bb37e,"""\nmore\ni can't make any real suggestions on ...",0,0,0,0,0,0,622,0,0,0,2,0,114,82,0.719298,0
0001d958c54c6e35,"you, sir, are my hero any chance you remember...",0,0,0,0,0,0,67,0,0,1,2,0,13,13,1.000000,0


In [133]:
X = df_train['comment_text']
Y = df_train.drop(['comment_text'], axis=1)

In [134]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

In [135]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8, test_size=0.2, random_state=23)

#### Vectorizing using Term frequency and inter document frequency 

In [136]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [137]:
tf_vectorizer = TfidfVectorizer(decode_error='ignore',  min_df=0.009,stop_words='english')

In [138]:
# X_test.drop(['vectorized'],axis=1)

In [139]:
X_train = tf_vectorizer.fit_transform(X_train)
X_test = tf_vectorizer.transform(X_test)

#### Learn to use oneVsRest multiclass classifier

In [140]:
# X_train = X_train.drop(['comment_text'],axis=1)
# X_test =X_test.drop(['comment_text'],axis=1)

In [141]:
# X_train = X_train.dropna(axis=1)
# X_test.isnull().sum()
# X_test = X_test.dropna(axis=1)#.sum()

In [142]:
from sklearn.multiclass import OneVsRestClassifier

In [143]:
from sklearn.svm import LinearSVC

In [144]:
model = OneVsRestClassifier(LinearSVC())
model.fit(X_train, Y_train)

OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [145]:
# Accuracy score on test dataset
model.score(X_test,Y_test)

0.9073476421745261

In [146]:
# Accuracy score on train dataset
model.score(X_train,Y_train)

0.9079009212257944

In [147]:
model.predict(X_test[0:10])

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0]])

In [21]:
Y_test[0:10]

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
88d640ab9223398d,0,0,0,0,0,0
9f9b412e0a99e6dc,0,0,0,0,0,0
323848f9f866c47e,0,0,0,0,0,0
90bd2ed36352d163,0,0,0,0,0,0
fe77e59403ac395e,0,0,0,0,0,0
17e22e51f52ae4a4,0,0,0,0,0,0
b1d80e1ab696b108,0,0,0,0,0,0
9329101468aa1b93,0,0,0,0,0,0
4c4cae190527b2a1,0,0,0,0,0,0


In [22]:
from sklearn.multiclass import OneVsOneClassifier

In [23]:
model_one_v_one = OneVsOneClassifier(LinearSVC())

In [24]:
# model_one_v_one.fit(X_train,Y_train)
# model_one_v_one.score(X_test, Y_test)

###  Using Multiple Output regressor -

Multiple out regressor fits multiple regressor - this stratergy fits one regressor per target. Since each traget is represented by exactly one regressor  it is possible to drive strength out of it.

In [25]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [26]:
multi_model = MultiOutputClassifier(GradientBoostingClassifier())

In [27]:
%%time
multi_model.fit(X_train, Y_train)


Wall time: 6min 40s


MultiOutputClassifier(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
           n_jobs=1)

In [28]:
multi_model.score(X_test, Y_test)

0.9054989816700612

In [29]:
multi_model.score(X_train, Y_train)

0.9103214890016921

### Checking the score of different classifiers

In [76]:
from sklearn import linear_model, svm, tree, ensemble

models = [
#     linear_model.LogisticRegressionCV(cv =5),
#     linear_model.RidgeClassifier(),
    OneVsRestClassifier(linear_model.LogisticRegression(solver= 'newton-cg')),
    OneVsRestClassifier(linear_model.LogisticRegression(solver= 'sag')),

    OneVsRestClassifier(linear_model.RidgeClassifier(solver= 'sag')),
    OneVsRestClassifier(linear_model.SGDClassifier()),
#     linear_model.RidgeClassifier(solver= 'newton-cg'),

]
#     linear_model.RidgeClassifierCV(),
#     linear_model.Lasso(),
#     linear_model.ElasticNet(),
#     linear_model.BayesianRidge(),
# #     linear_model.RANSACRegressor(),
#     svm.LinearSVR(),
#     svm.SVR(),
#     tree.DecisionTreeRegressor(),
#     tree.ExtraTreeRegressor(),
#     ensemble.RandomForestRegressor(),
#     ensemble.GradientBoostingRegressor()

# ]

# Ridge classifier does not support multiclass classifications

In [204]:
for model in models:
    model.fit(X_train,Y_train)
    print('-'*30)
    print(model.__class__.__name__)
#     print(model.classes_)
#     print(model.estimators_)
    
    train_pred = model.predict(X_train)
    train_rmse = np.sqrt(mean_squared_error(Y_train, train_pred))
    print("Root mean square error train {}".format(train_rmse))    
    print("Accuracy score for traing set {}".format(model.score(X_train,Y_train)))

    pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(Y_test, pred))
    print("Root mean square error test {}".format(rmse))
    print("Accuracy score for test set {}".format(model.score(X_test,Y_test)))

------------------------------
OneVsRestClassifier
Root mean square error train 0.16238547930398348
Accuracy score for traing set 0.9075797455662092
Root mean square error test 0.16298882137514256
Accuracy score for test set 0.9070343098856337


C:\Users\amitkumar_kataria\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\amitkumar_kataria\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\amitkumar_kataria\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\amitkumar_kataria\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users

------------------------------
OneVsRestClassifier
Root mean square error train 0.19082181218989785
Accuracy score for traing set 0.897952309331328
Root mean square error test 0.19193080202018906
Accuracy score for test set 0.8965690114366286
------------------------------
OneVsRestClassifier
Root mean square error train 0.19093467088807392
Accuracy score for traing set 0.8985711599924798
Root mean square error test 0.19218911169885664
Accuracy score for test set 0.8971330095566348


C:\Users\amitkumar_kataria\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\amitkumar_kataria\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will

------------------------------
OneVsRestClassifier
Root mean square error train 0.19833022753577959
Accuracy score for traing set 0.8841104217584759
Root mean square error test 0.19978316095225118
Accuracy score for test set 0.8820303932320226


###  Getting into Feature Engineering
As we see above we have trained 4 different models but our accuracy is not going above 90% to enhance the accuracy we are going to create some new features

In [151]:
print(X_train.shape)
print(type(X_train))
print(df_train.columns)

(127656, 455)
<class 'scipy.sparse.csr.csr_matrix'>
Index(['comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate', 'total_length', 'capitals', 'num_exclamation_marks',
       'num_question_marks', 'num_punctuation', 'num_symbols', 'num_words',
       'num_unique_words', 'words_vs_unique', 'num_smilies'],
      dtype='object')


In [173]:
df_train['total_length'] = df_train['comment_text'].apply(len)
df_train['capitals'] = df_train['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
# # df_train['caps_vs_length'] = df_train.apply(lambda row: float(row['capitals'])/float(row['total_length']),
# #                                 axis=1)
df_train['num_exclamation_marks'] = df_train['comment_text'].apply(lambda comment: comment.count('!'))
df_train['num_question_marks'] = df_train['comment_text'].apply(lambda comment: comment.count('?'))
df_train['num_punctuation'] = df_train['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in '.,;:'))
df_train['num_symbols'] = df_train['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in '*&$%'))
df_train['num_words'] = df_train['comment_text'].apply(lambda comment: len(comment.split()))
df_train['num_unique_words'] = df_train['comment_text'].apply(
    lambda comment: len(set(w for w in comment.split())))
df_train['words_vs_unique'] = df_train['num_unique_words'] / df_train['num_words']
df_train['num_smilies'] = df_train['comment_text'].apply(
    lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))

In [158]:
from scipy.sparse import csr_matrix,vstack

In [174]:
X = df_train[['comment_text',  'total_length', 'capitals', 'num_exclamation_marks',
       'num_question_marks', 'num_punctuation', 'num_symbols', 'num_words',
       'num_unique_words', 'words_vs_unique', 'num_smilies']]
Y = df_train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate',]]

In [175]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8, test_size=0.2, random_state=23)

In [171]:
# X_train.comment_text

In [176]:
coments_X_train = tf_vectorizer.fit_transform(X_train['comment_text'])
comments_X_test = tf_vectorizer.transform(X_test['comment_text'])

In [178]:
other_featuers_spars_X_train = csr_matrix(X_train[['total_length', 'capitals', 'num_exclamation_marks',
       'num_question_marks', 'num_punctuation', 'num_symbols', 'num_words',
       'num_unique_words', 'words_vs_unique', 'num_smilies']])

other_featuers_spars_X_test = csr_matrix(X_test[['total_length', 'capitals', 'num_exclamation_marks',
       'num_question_marks', 'num_punctuation', 'num_symbols', 'num_words',
       'num_unique_words', 'words_vs_unique', 'num_smilies']])

In [191]:
# combined = hstack([other_spares,X_train])
# print(other_spares.shape)
print(coments_X_train.shape)
print(other_featuers_spars_X_train.shape)
print(X_train.shape)
# print(coments_X_train.isnull().sum())
# print(other_featuers_spars_X_train.isnull().sum())
# print(X_train.isnull().sum())
# 
# print(other_spares.__class__)
# print(combined.shape)

(127656, 455)
(127656, 10)
(127656, 465)


In [203]:
np.isnan(X_train.data).any()
# other_featuers_spars_X_test.data = np.nan_to_num(other_featuers_spars_X_test.data)

False

In [202]:
X_train = hstack([coments_X_train, other_featuers_spars_X_train])
X_test = hstack([comments_X_test, other_featuers_spars_X_test])

In [190]:
X_train.dtype

dtype('float64')

### Test only for - UnderStanding TfidfVectorizer
NOt the part of acutal solution

In [30]:
x_df_train = df_train['comment_text']

x_df_train.isnull().any()

False

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [32]:

tf_vectorizer = TfidfVectorizer(decode_error='ignore',  min_df=0.009,stop_words='english')

In [33]:
trans = tf_vectorizer.fit_transform(x_df_train)

In [34]:
print(trans.shape)
print(tf_vectorizer.get_feature_names())
print(len(tf_vectorizer.get_feature_names()))

(159571, 454)
['10', '100', '11', '12', '14', '15', '20', '2005', '2006', '2007', '2008', '24', 'able', 'according', 'account', 'actual', 'actually', 'add', 'added', 'adding', 'address', 'admin', 'administrator', 'admins', 'ago', 'agree', 'allowed', 'american', 'answer', 'anti', 'appear', 'appears', 'appreciate', 'appropriate', 'aren', 'argument', 'article', 'articles', 'ask', 'asked', 'attack', 'attacks', 'attention', 'automatically', 'aware', 'away', 'bad', 'based', 'believe', 'best', 'better', 'big', 'bit', 'block', 'blocked', 'book', 'books', 'called', 'came', 'care', 'case', 'category', 'certain', 'certainly', 'change', 'changed', 'changes', 'check', 'cheers', 'cited', 'claim', 'claims', 'clear', 'clearly', 'come', 'comment', 'comments', 'common', 'community', 'completely', 'consensus', 'consider', 'considered', 'contact', 'content', 'continue', 'contribs', 'contributing', 'contributions', 'copyright', 'correct', 'country', 'course', 'create', 'created', 'criteria', 'current', 'cu

In [35]:
print(tf_vectorizer.get_params())

{'analyzer': 'word', 'binary': False, 'decode_error': 'ignore', 'dtype': <class 'numpy.int64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 0.009, 'ngram_range': (1, 1), 'norm': 'l2', 'preprocessor': None, 'smooth_idf': True, 'stop_words': 'english', 'strip_accents': None, 'sublinear_tf': False, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'use_idf': True, 'vocabulary': None}


In [36]:
print(tf_vectorizer.get_stop_words())

frozenset({'call', 'became', 'few', 'to', 'are', 'although', 'anywhere', 'above', 'even', 'for', 'system', 'herself', 'thereupon', 'whose', 'yourselves', 'every', 'whoever', 'how', 'or', 'off', 'whereby', 'interest', 'keep', 'out', 'bill', 'de', 'name', 'should', 'sincere', 'fire', 'nobody', 'same', 'nor', 'her', 'cry', 'un', 'not', 'can', 'eight', 'next', 'on', 'serious', 'several', 'though', 'whole', 'yourself', 'get', 'others', 'hundred', 'anyone', 'now', 'meanwhile', 're', 'empty', 'give', 'between', 'and', 'down', 'everyone', 'somehow', 'through', 'also', 'than', 'am', 'very', 'ever', 'put', 'he', 'seeming', 'behind', 'last', 'latterly', 'someone', 'made', 'be', 'third', 'thru', 'within', 'only', 'whatever', 'cant', 'again', 'anything', 'still', 'but', 'into', 'is', 'where', 'four', 'formerly', 'however', 'least', 'side', 'ten', 'as', 'enough', 'six', 'top', 'after', 'while', 'will', 'against', 'elsewhere', 'thus', 'move', 'amongst', 'across', 'somewhere', 'hers', 'one', 'per', 'w

### Summary -- Take Aways


#### References :
https://www.kaggle.com/eikedehling/feature-engineering

[Multi Class Classification](# https://www.youtube.com/watch?v=y0wQK3mnrNY)